Cryptocompare  
https://developers.cryptocompare.com/documentation/data-api/spot_v1_historical_hours
biance,bybit,okx,bitget,coinbase,kraken,upbit,gateio,bitfinex,gemini,binanceusa,bingx,bitmex,bitstamp,coinbaseinternational,kucoin,

In [1]:
import os
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "83b1fe1bf4762e0bfb056afa64f89b227988cd131d26ee0ff465b9c3a7070e19" # cryptocompare api key

In [2]:
url_markets = "https://data-api.cryptocompare.com/spot/v1/markets"

headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}
response = requests.get(url_markets, headers=headers)

if response.status_code == 200:
    data = response.json()["Data"]

    # **提取所有 market（交易所）**
    markets = [{"market": market, **details} for market, details in data.items()]
    markets_df = pd.DataFrame(markets)
    print(f"✅ 获取 {len(markets_df)} 个交易所")

else:
    print(f"❌ API 请求失败: {response.status_code}, {response.text}")

markets_df.head()

✅ 获取 176 个交易所


,market,TYPE,EXCHANGE_STATUS,MAPPED_INSTRUMENTS_TOTAL,UNMAPPED_INSTRUMENTS_TOTAL,INSTRUMENT_STATUS,TOTAL_TRADES_SPOT,HAS_ORDERBOOK_L2_MINUTE_SNAPSHOTS_ENABLED
0,aax,602,ACTIVE,250,185,"{'ACTIVE': 88, 'IGNORED': 0, 'RETIRED': 347, '...",648891006,False
1,abcc,602,ACTIVE,304,28,"{'ACTIVE': 29, 'IGNORED': 0, 'RETIRED': 303, '...",8437466,False
2,acx,602,ACTIVE,0,32,"{'ACTIVE': 32, 'IGNORED': 0, 'RETIRED': 0, 'EX...",6203612,False
3,aidosmarket,602,ACTIVE,1,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 1, 'EXP...",155608,False
4,alphaex,602,ACTIVE,10,2,"{'ACTIVE': 2, 'IGNORED': 0, 'RETIRED': 10, 'EX...",1718824,False


In [10]:
selected_markets = [
    "binance", "binanceaggregate", "binanceusa", "bingx", "bitfinex",
    "bitget", "bitmex", "bitstamp", "bybit", "coinbase",
    "coinbaseinternational", "gateio", "gemini", "kraken", "kucoin",
    "okx", "upbit"
]

In [11]:
url_instruments = "https://data-api.cryptocompare.com/spot/v1/markets/instruments"
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

all_instruments = []

for market in selected_markets:
    params = {"market": market}
    response = requests.get(url_instruments, headers=headers, params=params)

    json_response = response.json()
    market_data = json_response["Data"].get(market, {})
    instruments_data = market_data.get("instruments", {})

    # 遍历market的所有交易对
    for instrument_name, instrument_details in instruments_data.items():
        all_instruments.append({
            "market": market,
            "instrument": instrument_name,
            "base_asset": instrument_details.get("INSTRUMENT", "N/A"),
            "status": instrument_details.get("INSTRUMENT_STATUS", "UNKNOWN")
        })

    print(f"✅ {market} 交易所的 {len(instruments_data)} 个交易对已获取")

    # 避免过快请求
    time.sleep(0.5)

instruments_df = pd.DataFrame(all_instruments)
instruments_df.head()

save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
csv_filepath = os.path.join(save_dir, "crypto_instruments.csv")
instruments_df.to_csv(csv_filepath, index=False, encoding="utf-8")


✅ binance 交易所的 1344 个交易对已获取
✅ binanceaggregate 交易所的 1344 个交易对已获取
✅ binanceusa 交易所的 183 个交易对已获取
✅ bingx 交易所的 883 个交易对已获取
✅ bitfinex 交易所的 285 个交易对已获取
✅ bitget 交易所的 806 个交易对已获取
✅ bitmex 交易所的 16 个交易对已获取
✅ bitstamp 交易所的 244 个交易对已获取
✅ bybit 交易所的 610 个交易对已获取
✅ coinbase 交易所的 432 个交易对已获取
✅ coinbaseinternational 交易所的 2 个交易对已获取
✅ gateio 交易所的 2579 个交易对已获取
✅ gemini 交易所的 102 个交易对已获取
✅ kraken 交易所的 867 个交易对已获取
✅ kucoin 交易所的 1080 个交易对已获取
✅ okx 交易所的 0 个交易对已获取
✅ upbit 交易所的 429 个交易对已获取


In [14]:
instrument_filepath = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result/crypto_instruments.csv"
instruments_df = pd.read_csv(instrument_filepath)

url_spot_volume = 'https://data-api.cryptocompare.com/spot/v1/historical/hours'
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"

# **遍历所有交易所**
for market in instruments_df["market"].unique():
    trade_data = []
    
    # **筛选当前交易所的交易对**
    market_instruments_df = instruments_df[instruments_df["market"] == market]

    for _, row in market_instruments_df.iterrows():
        instrument = row["instrument"]
        
        params = {
            "market": market,
            "instrument": instrument,
            "limit": 2000,  # 获取最近 2000 小时数据
            "aggregate": 1,  # 每一个小时
            "fill": "true",
            "apply_mapping": "true",
            "response_format": "JSON",
        }

        response = requests.get(url_spot_volume, headers=headers, params=params)
        
        if response.status_code == 200:
            json_response = response.json()

            # **检查数据是否有效**
            if "Data" in json_response and isinstance(json_response["Data"], list) and len(json_response["Data"]) > 0:
                for entry in json_response["Data"]:  # 遍历 2000 小时数据
                    trade_data.append({
                        "market": market,
                        "instrument": instrument,
                        "base_asset": entry["BASE"],  # 基础资产
                        "quote_asset": entry["QUOTE"],  # 报价资产
                        "time": pd.to_datetime(entry["TIMESTAMP"], unit="s"),  # 转换时间
                        "open": entry["OPEN"],  # 开盘价
                        "high": entry["HIGH"],  # 最高价
                        "low": entry["LOW"],  # 最低价
                        "close": entry["CLOSE"],  # 收盘价
                        "total_trades": entry["TOTAL_TRADES"],  # 总交易笔数
                        "total_trades_buy": entry["TOTAL_TRADES_BUY"],  # 买单数
                        "total_trades_sell": entry["TOTAL_TRADES_SELL"],  # 卖单数
                        "volume": entry["VOLUME"],  # 以 base_asset 计数的交易量
                        "quote_volume": entry["QUOTE_VOLUME"],  # 以 quote_asset 计数的交易量
                        "volume_buy": entry["VOLUME_BUY"],  # 以 base_asset 计数的买单交易量
                        "quote_volume_buy": entry["QUOTE_VOLUME_BUY"],  # 以 quote_asset 计数的买单交易量
                        "volume_sell": entry["VOLUME_SELL"],  # 以 base_asset 计数的卖单交易量
                        "quote_volume_sell": entry["QUOTE_VOLUME_SELL"],  # 以 quote_asset 计数的卖单交易量
                    })
                print(f"✅ Data successfully fetched: {market} - {instrument} ({len(json_response['Data'])} hours)")
            else:
                print(f"❌ No data for {market} - {instrument}")

        else:
            print(f"❌ API request failed for {market} - {instrument}, Status Code: {response.status_code}")

        time.sleep(0.5)

    # **转换成 DataFrame**
    if trade_data:
        spot_volume_df = pd.DataFrame(trade_data)
        
        # **存储 CSV，按交易所划分**
        csv_filepath = os.path.join(save_dir, f"{market}_spot_volume_2000h.csv")
        spot_volume_df.to_csv(csv_filepath, index=False, encoding="utf-8")
        print(f"📁 Data for {market} saved to {csv_filepath}")


✅ Data successfully fetched: binance - MANA-USDT (2000 hours)
✅ Data successfully fetched: binance - VOXEL-USDT (2000 hours)
✅ Data successfully fetched: binance - AUDIO-BTC (2000 hours)
✅ Data successfully fetched: binance - JST-BTC (2000 hours)
✅ Data successfully fetched: binance - MBL-USDT (2000 hours)
✅ Data successfully fetched: binance - BEL-TRY (2000 hours)
✅ Data successfully fetched: binance - APT-BTC (2000 hours)
✅ Data successfully fetched: binance - PAXG-BTC (2000 hours)
✅ Data successfully fetched: binance - ETH-BTC (2000 hours)
✅ Data successfully fetched: binance - QKC-USDT (2000 hours)
✅ Data successfully fetched: binance - MOVR-TRY (2000 hours)
✅ Data successfully fetched: binance - BLUR-USDT (2000 hours)
✅ Data successfully fetched: binance - LSK-BTC (2000 hours)
✅ Data successfully fetched: binance - VIC-TRY (2000 hours)
✅ Data successfully fetched: binance - FIL-FDUSD (2000 hours)
✅ Data successfully fetched: binance - ANKR-TRY (2000 hours)
✅ Data successfully fetc

KeyboardInterrupt: 

In [ ]:
pivot_df = spot_volume_df.pivot(index="instrument", columns="market", values="total_trades")

In [15]:
# **选择一个交易所 & 交易对**
market = "binance"
instrument = "BTC-USDT"

# **CryptoCompare API 端点**
url_spot_volume = 'https://data-api.cryptocompare.com/spot/v1/historical/hours'
headers = {"Content-type": "application/json", "authorization": f"Apikey {API_KEY}"}

params = {
    "market": market,
    "instrument": instrument,
    "limit": 2000,  # 获取最近 2000 小时数据
    "aggregate": 1,  # 每一个小时
    "fill": "true",
    "apply_mapping": "true",
    "response_format": "JSON",
}

response = requests.get(url_spot_volume, headers=headers, params=params)

if response.status_code == 200:
    json_response = response.json()

    # **检查数据是否有效**
    if "Data" in json_response and isinstance(json_response["Data"], list) and len(json_response["Data"]) > 0:
        trade_data = []

        for entry in json_response["Data"]:  # 遍历 2000 小时数据
            trade_data.append({
                "market": market,
                "instrument": instrument,
                "base_asset": entry["BASE"],  # 基础资产
                "quote_asset": entry["QUOTE"],  # 报价资产
                "time": pd.to_datetime(entry["TIMESTAMP"], unit="s"),  # 转换时间
                "open": entry["OPEN"],  # 开盘价
                "high": entry["HIGH"],  # 最高价
                "low": entry["LOW"],  # 最低价
                "close": entry["CLOSE"],  # 收盘价
                "total_trades": entry["TOTAL_TRADES"],  # 总交易笔数
                "total_trades_buy": entry["TOTAL_TRADES_BUY"],  # 买单数
                "total_trades_sell": entry["TOTAL_TRADES_SELL"],  # 卖单数
                "volume": entry["VOLUME"],  # 以 base_asset 计数的交易量
                "quote_volume": entry["QUOTE_VOLUME"],  # 以 quote_asset 计数的交易量
                "volume_buy": entry["VOLUME_BUY"],  # 以 base_asset 计数的买单交易量
                "quote_volume_buy": entry["QUOTE_VOLUME_BUY"],  # 以 quote_asset 计数的买单交易量
                "volume_sell": entry["VOLUME_SELL"],  # 以 base_asset 计数的卖单交易量
                "quote_volume_sell": entry["QUOTE_VOLUME_SELL"],  # 以 quote_asset 计数的卖单交易量
            })

        print(f"✅ Data successfully fetched: {market} - {instrument} ({len(json_response['Data'])} hours)")

        # **转换成 DataFrame**
        trade_volume_df = pd.DataFrame(trade_data)

        # **存储 CSV**
        save_dir = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/spot_volume/result"
        os.makedirs(save_dir, exist_ok=True)  # 确保文件夹存在
        csv_filepath = os.path.join(save_dir, f"{market}_{instrument}_spot_volume_2000h.csv")
        trade_volume_df.to_csv(csv_filepath, index=False, encoding="utf-8")

        # **显示前 10 行的数据**
    else:
        print(f"❌ No data for {market} - {instrument}")

else:
    print(f"❌ API request failed for {market} - {instrument}, Status Code: {response.status_code}")


✅ Data successfully fetched: binance - BTC-USDT (2000 hours)
